### Import Libraries

In [91]:

import numpy as np
import pandas as pd

import os
import nltk
import gensim

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from gensim.test.utils import common_texts, get_tmpfile

In [90]:
wv_temp=wv

In [92]:
path = get_tmpfile("wordvectors.kv")
wv.save("word2vec.model")


: 

In [ ]:
model = Word2Vec.load("word2vec.model")
model??

In [56]:
len(wv.index_to_key)

3000000

### Load datas

In [57]:
pd.set_option('display.max_colwidth',100)

messages = pd.read_csv('spam.csv', encoding='latin-1')
messages = messages.drop(labels=["Unnamed: 2","Unnamed: 3","Unnamed: 4"], axis = 1)
messages.columns = ["label","text"]
messages.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [58]:
drop_random=np.random.choice(messages[messages['label']=='ham'].index,size=4000,replace=False)
messages=messages.drop(drop_random)
print(messages.groupby('label').count())


       text
label      
ham     825
spam    747


In [59]:
messages['text_clean'] = messages['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
#messages['text_list'] = messages['text'].apply(lambda x: x.split(" "))
messages.head()

,label,text,text_clean
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,"[free, entry, in, wkly, comp, to, win, fa, cup, final, tkts, st, may, text, fa, to, to, receive,..."
5,spam,FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for ...,"[freemsg, hey, there, darling, it, been, week, now, and, no, word, back, like, some, fun, you, u..."
8,spam,WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To ...,"[winner, as, valued, network, customer, you, have, been, selected, to, receivea, prize, reward, ..."
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with came...,"[had, your, mobile, months, or, more, entitled, to, update, to, the, latest, colour, mobiles, wi..."
11,spam,"SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, ...","[six, chances, to, win, cash, from, to, pounds, txt, csh, and, send, to, cost, day, days, tsandc..."


### Encoding the label column

In [60]:
messages['label'] = messages['label'].map({'ham':1,'spam':0})


### Split data into train and test sets

In [61]:
X_train, X_test, y_train,y_test = train_test_split(messages['text_clean'],messages['label'],test_size=0.2)

In [62]:
words = set(wv.index_to_key )
X_train_vect = np.array([np.array([wv[i] for i in ls if i in words])
                         for ls in X_train],dtype=object)
X_test_vect = np.array([np.array([wv[i] for i in ls if i in words])
                         for ls in X_test],dtype=object)

### Why is the length of the sentence different than the lemgth of the sentence vector?

In [63]:
for i,v in enumerate(X_train_vect):
    print(len(X_train.iloc[i]),len(v))
    if i>10:
        break

18 15
14 12
29 24
11 11
9 9
11 10
16 14
22 18
20 17
24 20
4 4
15 13


### Compute sentence vectors by avergaing the word vectors for the words contained in the sentence

In [64]:

X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(300, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(300, dtype=float))

#### Are our sentence vector lengths consistent?

In [65]:
for i,v in enumerate(X_train_vect_avg):
    print(len(X_train.iloc[i]),len(v))
    if i > 10:
        break

18 300
14 300
29 300
11 300
9 300
11 300
16 300
22 300
20 300
24 300
4 300
15 300


#### Fit RandomForestClassifier On Top of Word Vectors

In [66]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg,y_train.values.ravel())

#### Prediction

In [67]:
y_pred = rf_model.predict(X_test_vect_avg)

In [68]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision: 0.933 / Recall: 0.974 / Accuracy: 0.952


#### SVC

In [69]:
svc = SVC()
svc_model = rf.fit(X_train_vect_avg,y_train.values.ravel())

In [70]:
y_pred = svc_model.predict(X_test_vect_avg)


In [71]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision: 0.928 / Recall: 0.994 / Accuracy: 0.959


In [72]:
sentence_test = "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"
#sentence_test = "Nah I don't think he goes to usf, he lives around here though"
sentence_test =  gensim.utils.simple_preprocess(sentence_test)
words = set(wv.index_to_key )
sentence_test_vect = np.array([np.array([wv[i] for i in sentence_test if i in words])])

sentence_test_vect_avg = []
for v in sentence_test_vect:
    if v.size:
        sentence_test_vect_avg.append(v.mean(axis=0))
    else:
        sentence_test_vect_avg.append(np.zeros(100, dtype=float))
rf.predict(sentence_test_vect_avg)

array([0], dtype=int64)

In [73]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

[[147  12]
 [  1 155]]


### Hyper Tuning

In [75]:
svc=SVC()
knn=KNeighborsClassifier()
dtc=DecisionTreeClassifier()
mb=MultinomialNB()
rtc=RandomForestClassifier()

In [79]:
models={'svc':svc,'knn':knn,'dtc':dtc,'rtc':rtc}


In [80]:
params={
    'knn': 
        {'n_neighbors':[3,5,7,9,11,13,15],
        'metric':['cosine','euclidean','manhattan'],
        'weights':['uniform','distance']},
    
    'svc': {'C':[0.1,1,10,100], 
            'gamma':[1,0.1,0.01,0.01], 
            'kernel':['rbf','linear']},
    
    'dtc':{
        'criterion':['gini','entropy'],
        'max_depth':[2,4,6,8,10,12]
    },
    
    'mb':{
        'alpha': [1.0,2.0],
    'fit_prior': [True]
    },
    'rtc':{
        'criterion':['gini','entropy'],
        'max_depth':[2,4,6,8,10,12]
    }
}

In [81]:
model_accuracy={}
score=0.0001
for model in models.keys():
    mod = GridSearchCV(
    models[model],
    params[model],
    verbose=0, #Progress bar showing
    cv=5, #cross validation
    n_jobs=-1, #cores to assign
)
    gridsearch_result=mod.fit(X_train_vect_avg,y_train.values.ravel())
    
    #selects best model
    if(score < float(gridsearch_result.score(X_test_vect_avg,y_test))):
            score=gridsearch_result.score(X_test_vect_avg,y_test)
            best_model=gridsearch_result
            
    predict=mod.predict(X_test_vect_avg)
    print(f"{model} : ",gridsearch_result.best_estimator_)
    if model not in model_accuracy.keys():
        model_accuracy.update({model:accuracy_score(y_test,predict)})

svc :  SVC(C=10, gamma=1)
knn :  KNeighborsClassifier(metric='cosine', n_neighbors=3, weights='distance')
dtc :  DecisionTreeClassifier(criterion='entropy', max_depth=8)
rtc :  RandomForestClassifier(criterion='entropy', max_depth=8)


In [82]:
model_accuracy

{'svc': 0.9619047619047619,
 'knn': 0.9428571428571428,
 'dtc': 0.8920634920634921,
 'rtc': 0.9523809523809523}